# Tarea 3.2 - Agrupamiento - Algoritmos Genéticos

## Edison David González Blandón & Juan Esteban Velasquéz Franco

### Inicializando las condiciones iniciales y generando datos

In [ ]:
%matplotlib inline
from scipy.spatial import distance
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np

Fitness=[]
indice=[]        #Lista que contiene los indices de los clusters segun cada grupo
cromosomas=10     #Tamaño de la poblacion (filas) - Cromosomas
genes=16         #Numero de Genes
K=4               #Cluster

#Creación de los datos
mean = [5, 10]            
cov = [[1, 0], [0, 10]]  # diagonal covariance
x1, x2 = np.random.multivariate_normal(mean, cov, int(genes/2)).T
plt.plot(x1, x2, 'x')
X_C1 = np.concatenate((x1[:,None],x2[:,None]),axis=1)  #Concatena [x1,x2] y genera vector de tuplas

mean = [10, 10]
cov = [[1, 0], [0, 10]]  # diagonal covariance
x1, x2 = np.random.multivariate_normal(mean, cov, int(genes/2)).T
plt.plot(x1, x2, 'x')
plt.show()
X_C2 = np.concatenate((x1[:,None],x2[:,None]),axis=1)  #Concatena [x1,x2] y genera vector de tuplas

Xdata = np.concatenate((X_C1,X_C2),axis=0)
#plt.plot(Xdata[:,0], Xdata[:,1], 'x')
#plt.show()

### Generando la matriz población

In [ ]:
poblacion=[]
for j in range(cromosomas):
    poblacion.append(np.random.randint(1,K+1,genes))    #Genera vector de etiquetas Random
poblacion=np.array(poblacion)
#print(poblacion)
#print(np.shape(poblacion))    

### Enlazar datos con la matriz población

In [ ]:
def IndexEtiq(poblacion,K):
    indice=[]
    for j in range (0,len(poblacion)):
        for i in range(1,K+1):
            indice.append(np.where(poblacion[j,:]==i)[0])
            #print(indice)    #Contiene los indices de cada cluster
            #print(indice[0])
    arreglo_indices=np.array(indice)
    indices=indice
    indice=[]
    #print(indices)
    return indices

indices_final = IndicesEtiqueta(poblacion,K)

### Función Fitness - Función Objetivo

In [ ]:
def FuncionObjetivo(cromosomas,K,indices_final,Xdata):
    Fitness=[]
    for c in range(cromosomas):
        fitness_acum=0
        for g in range (K):
            eleccion_grupo=(c*K+g)
            for v1 in range (len(indices_final[eleccion_grupo])):
                for v2 in range (v1,len(indices_final[eleccion_grupo])):
                    a=Xdata[indices_final[eleccion_grupo][v1]]
                    b=Xdata[indices_final[eleccion_grupo][v2]]
                    fitness_acum+=distance.euclidean(a,b)
        #print(fitness_acum)
        fitness_total_cr = 1/fitness_acum
        Fitness.append(fitness_total_cr)
    #print(Fitness)
    return Fitness

### Funciones de Cruce

In [1]:
def CruceMultipunto(a,b):
    cruce1 = np.random.randint(0,len(a))#Primer punto de cruce
    cruce2 = 1+np.random.randint(cruce1,len(a))#Se le suma 1 para que el segundo cruce se haga al menos una posicion más allá 
    print("corte1: ",cruce1)
    print("corte2: ",cruce2-1)
    temp = np.zeros(len(a))
    temp[cruce1:cruce2] = b[cruce1:cruce2]
    b[cruce1:cruce2] = a[cruce1:cruce2]
    a[cruce1:cruce2]=temp[cruce1:cruce2]
    return a,b

### Funciones de Mutación

In [2]:
def Mutacion_RandomResetting(a,K):
    gen1_elegido=np.random.randint(0,len(a)) #Se escoge el gen para mutar aleatoriamente 
    print("gen elegido para mutar: ",gen1_elegido)
    resetting=np.random.randint(1,K+1)#mutación
    print("Resetting: ",resetting)
    a[gen1_elegido]=resetting
    #print("cromosoma 'a' mutado: ",a)
    return a

### Wheel - Ruleta ara elección de padres e implementación de los cruces y finalización de iteraccion por generaciones

#### Variables control, variación de generaciones

In [ ]:
Tm = 0.4                #Tasa de mutacion 
Tc = 0.4                #Tasa de cruce porcentaje de parejas van a ser seleccionadas para el cruce
N_gen = 4               #Numero de generaciones
alpha = 0.6             #For aritmetic recombination
mejor_fitnes = []       #Lista con los mejores fitness

In [ ]:
for gen in range(N_gen):
    
    #print("Poblacion Inicial")
    #print(poblacion)
    
    # Fitness evaluation
    Fitness = np.array(FuncionObjetivo(cromosomas,K,indices_final,Xdata)) #Primero hallando, el resultado de la función fitness
    #print(Fitness)
    BestIndividuos = np.argsort(-Fitness) #argsort, orden de los indices con mejores fitness
    
    #print("Poblacion Cambio 1")
    poblacion = poblacion[BestIndividuos]
    #print(poblacion)
    
    Wheel = np.cumsum(Fitness[BestIndividuos])/np.sum(Fitness[BestIndividuos])
    
    # Crossover - Wheel 
    
    for parent in range(int(Tc*cromosomas/2)):
        P = np.random.rand(1)
        ParentSelected_1 = np.where(Wheel >= P)[0][0]
        
        P = np.random.rand(1)
        ParentSelected_2 = np.where(Wheel >= P)[0][0]
        
        #Crossover - Cruce Multipunto 
        Child_1 ,Child_2 = CruceMultipunto(poblacion[ParentSelected_1],poblacion[ParentSelected_2]
        
        #Add to population
        poblacion = np.append(poblacion,Child_1[None,:],axis=0)
        poblacion = np.append(poblacion,Child_2[None,:],axis=0)
        
        #print(poblacion)
    
    # Mutation - Wheel
    for parent in range(int(Tm*cromosomas)):
        P = np.random.rand(1)
        ParentSelected = np.where(Wheel >= P)[0][0]
        
        #Add to population
        Child_1 = poblacion[ParentSelected] + np.random.rand(1)
        poblacion = np.append(poblacion,Child_1[None,:],axis=0)
        
        print("Poblacion cambio 3")
        
        print(poblacion)
        
    #Selection
    Fitness = np.array(FuncionObjetivo(cromosomas,K,indices_final,Xdata))
    
    print("Poblacion Final")
    
    print(poblacion)
    
    BestIndividuos = np.argsort(-Fitness)
    poblacion = poblacion[BestIndividuos[:cromosomas]]
    Fitness = Fitness[BestIndividuos]
    
    plt.plot(gen,Fitness[0],'.')
plt.show()